In [1]:
import numpy as np
import cv2 

In [26]:
# 이미지 경로 입력하면 이미지, 양눈 좌표 반환 함수

def eye_detect(path):
    
    # 눈탐지 모델
    eye_casecade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    # np.fromfile(바이너리(텍스트)파일or경로, dtype)
    ff = np.fromfile(path, np.uint8) # uint8:양수, 2^8개, 0~255
    
    # cv2.imencode(출력파일확장자, 합축할이미지, params) return 압축결과(bool), 압축된이미지
    # params : https://docs.opencv.org/3.4/d8/d6a/group__imgcodecs__flags.html#ga292d81be8d76901bff7988d18d2b42ac
    
    # cv2.imdecode(인코딩된배열, ImreadModes) return 이미지배열
    # ImreadModes : https://docs.opencv.org/3.4/d8/d6a/group__imgcodecs__flags.html#ga61d9b0126a3e57d9277ac48327799c80
    img = cv2.imdecode(ff, cv2.IMREAD_UNCHANGED)
    
    # 이미지 리사이즈
    img = cv2.resize(img, dsize=(0, 0), fx=1.0, fy=1.0, interpolation=cv2.INTER_LINEAR)
    
    # 이미지 컬러 설정
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    # 눈탐지
    eyes = eye_casecade.detectMultiScale(img)
    
    # 눈 좌우 결정
    if eyes[0][0]<=eyes[1][0]:
        x1, x2, x3, x4 = eyes[0]
        y1, y2, y3, y4 = eyes[1]
        
    else :
        x1, x2, x3, x4 = eyes[1]
        y1, y2, y3, y4 = eyes[0]
    
    # 눈이미지만 추출
    left=img[x2:x2+x4,x1:x1+x3]
    right=img[y2:y2+y4,y1:y1+y3]

    # img:원본이미지, left:왼눈, right:오른눈, [[x1, x2, x3, x4], [y1, y2, y3, y4]]:[왼눈좌표, 오른눈좌표]
    return img, left, right, [[x1, x2, x3, x4], [y1, y2, y3, y4]]

In [49]:
img, left, right, eyes = eye_detect('./Data/Image/03.jpg')

# # 왼눈 확인
# cv2.imshow('img', left)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # 오른눈 확인
# cv2.imshow('img', right)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # 원본이미지 확인
# cv2.imshow('img', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [50]:
# 이미지에서 눈 좌표
eyes

[[60, 76, 35, 35], [105, 76, 34, 34]]

In [51]:
img2, left2, right2, eyes2 = eye_detect('./Data/Image/02.jpg')


# cv2.imshow('img', left2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# cv2.imshow('img', right2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# cv2.imshow('img', img2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [65]:
# 눈 합성

# 눈의 왼쪽 위 모서리 기준 x1=x좌표, x2=y좌표, x3,x4=길이
x1, x2, x3, x4=eyes[0]
y1, y2, y3, y4=eyes[1]

# 왼눈
# left2 = cv2.resize(left2, (0, 0), fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
left2 = cv2.resize(left2, eyes[1][2:])

# mask_l = left2[:, :, -1]    # mask는 알파 채널로 만든 마스크 영상
# left2 = left2[:, :, 0:3]    # src는 b, g, r 3채널로 구성된 컬러 영상

# img[y2:y2+y4,y1:y1+y3] =left2*1 + (img[y2:y2+y4,y1:y1+y3])*0.0


# 오른눈

# right2 = cv2.resize(right2, (0, 0), fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
right2 = cv2.resize(right2, eyes[0][2:])

# mask_r = right2[:, :, -1]    # mask는 알파 채널로 만든 마스크 영상
#right2 = right2[:, :, 0:3]    # src는 b, g, r 3채널로 구성된 컬러 영상

# img[x2:x2+x4,x1:x1+x3] =right2*1 + (img[x2:x2+x4,x1:x1+x3])*0.0


# cv2.imshow('dst', img)
# cv2.waitKey() 
# cv2.destroyAllWindows()

In [68]:
# 오른쪽 눈 합성
result = cv2.seamlessClone(
            right2,
            img,
            np.full(right2.shape[:2], 255, right2.dtype),
            (x1+(x3//2),x2+(x3//2)),
            cv2.MIXED_CLONE
        )

# 왼쪽 눈 합성
result = cv2.seamlessClone(
            left2,
            result,
            np.full(left2.shape[:2], 255, left2.dtype),
            (y1+(y3//2),y2+(y3//2)),
            cv2.MIXED_CLONE
        )

In [69]:
# 이미지 확인
cv2.imshow('after', result)
cv2.waitKey() 

-1